# Homework Assignment 4: Recipe Bot Retrieval Evaluation

This notebook shows you how to run the fourth homework example using Galileo. This homework involves building and evaluating a BM25 retrieval system for recipes.

## Configuration

To be able to run this notebook, you need to have a Galileo account set up, along with an LLM integration to run an experiment to generate responses.

1. If you don't have a Galileo account, head to [app.galileo.ai/sign-up](https://app.galileo.ai/sign-up) and sign up for a free account
1. Once you have signed up, you will need to configure an LLM integration. Head to the [integrations page](https://app.galileo.ai/settings/integrations) and configure your integration of choice. The notebook assumes you are using OpenAI, but has details on what to change if you are using a different LLM.
1. Create a Galileo API key from the [API keys page](https://app.galileo.ai/settings/api-keys)
1. In this folder is an example `.env` file called `.env.example`. Copy this file to `.env`, and set the value of `GALILEO_API_KEY` to the API key you just created.
1. If you are using a custom Galileo deployment inside your organization, then set the `GALILEO_CONSOLE_URL` environment variable to your console URL. If you are using [app.galileo.ai](https://app.galileo.ai), such as with the free tier, then you can leave this commented out.


In [ ]:
# Install the galileo and python-dotenv package into the current Jupyter kernel
%pip install "galileo" python-dotenv litellm rank-bm25

## Environment setup

To use Galileo, we need to load the API key from the .env file

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Check that the GALILEO_API_KEY environment variable is set
if not os.getenv("GALILEO_API_KEY"):
    raise ValueError("GALILEO_API_KEY environment variable is not set. Please set it in your .env file.")

Next we need to ensure there is a Galileo project set up.

In [ ]:
from galileo.projects import create_project, get_project

PROJECT_NAME = "AI Evals Course - Homework 4"
project = get_project(name=PROJECT_NAME)
if project is None:
    project = create_project(name=PROJECT_NAME)

print(f"Using project: {project.name} (ID: {project.id})")

## Step 1: Look at your data

We'll start by loading the datasets from GitHub and exploring a few rows.

First we load the recipes.

In [ ]:
import json
from urllib.request import urlopen

# Load the source data from the GitHub repository
processed_recipes_source_path = "https://raw.githubusercontent.com/ai-evals-course/recipe-chatbot/refs/heads/main/homeworks/hw4/reference_files/processed_recipes.json"

with urlopen(processed_recipes_source_path) as resp:
    recipes = json.load(resp)

print(f"Loaded {len(recipes)} recipes")

We can print the first one to see the data.

In [ ]:
# Look at one recipe
recipe = recipes[0]
print(f"Name: {recipe['name']}")
print(f"Cooking time: {recipe['minutes']} minutes")
print(f"Ingredients: {recipe['ingredients'][:5]}...")  # First 5
print(f"Steps: {len(recipe['steps'])} steps")

Next we load the synthetic queries.

In [ ]:
synthetic_queries_source_path = "https://raw.githubusercontent.com/ai-evals-course/recipe-chatbot/refs/heads/main/homeworks/hw4/reference_files/synthetic_queries.jsonl"

with urlopen(synthetic_queries_source_path) as resp:
    lines = (ln.decode("utf-8") for ln in resp)
    queries = [json.loads(line) for line in lines]

print(f"Loaded {len(queries)} synthetic queries")

Again we can print the first one to see the data.

In [ ]:
# Look at one query
q = queries[0]
print(f"Query: {q['query']}")
print(f"\nSource recipe: {q['source_recipe_name']}")
print(f"Source recipe ID: {q['source_recipe_id']}")
print(f"\nSalient fact (what makes this query answerable):\n{q['salient_fact'][:300]}...")

## Step 2: Build BM25 Retriever

Now we can build the BM25 retriever in the same way as the original homework.

In [ ]:
def recipe_to_text(recipe: dict) -> str:
    """Combine recipe fields into searchable text."""
    parts = [
        recipe['name'],
        ' '.join(recipe.get('ingredients', [])),
        ' '.join(recipe.get('steps', [])),
        ' '.join(recipe.get('tags', []))
    ]
    return ' '.join(parts).lower()

# Create corpus
corpus_texts = [recipe_to_text(r) for r in recipes]
print(f"Example text (first 300 chars):\n{corpus_texts[0][:300]}...")

In [ ]:
from rank_bm25 import BM25Okapi

# Simple tokenization (split on whitespace)
tokenized_corpus = [text.split() for text in corpus_texts]

# Build BM25 index
bm25 = BM25Okapi(tokenized_corpus)
print(f"BM25 index built with {len(tokenized_corpus)} documents")

In [ ]:
def retrieve(query: str, top_k: int = 5) -> list[tuple[dict, float]]:
    """Retrieve top-k recipes for a query.
    
    Returns: List of (recipe_index, score, recipe_name)
    """
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    
    # Get top-k indices
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    
    results = []
    for idx in top_indices:
        results.append((recipes[idx], scores[idx]))
    return results

In [ ]:
# Test it
test_query = "air fryer chicken crispy"
results = retrieve(test_query, top_k=5)

print(f"Query: {test_query}\n")
print("Top 5 results:")
for i, (recipe, score) in enumerate(results, 1):
    print(f"  {i}. {recipe['name']} (score: {score:.2f})")

## Step 2 additional - build a simple chatbot app that uses the retriever and logs to Galileo

Evals make more sense in terms of measuring part of an AI interaction. We should just score the `retrieve` function in code, but that doesn't simulate measuring a real world interaction. Instead, let's build a chat function with an LLM that retrieves data from the `retrieve` function to pass to the LLM, simulating a RAG system.

In [ ]:
from datetime import datetime

from galileo import galileo_context
from galileo.config import GalileoPythonConfig

def process_query(query: str) -> str:
    """
    Process the users query by retrieving relevant recipes and generating a response using the LLM.
    """
    print(f"Processing query: {query}")

    logger = galileo_context.get_logger_instance()

    # Capture the current time in nanoseconds for logging
    start_time_ns = datetime.now().timestamp() * 1_000_000_000

    # Get the results from the RAG system
    results = retrieve(query, top_k=5)

    # Log the retrieved recipes to the trace, along all the details of the query
    # To act as ground truth for evaluation later
    logger.add_retriever_span(
        name="BM25 Retriever",
        input=query,
        output=[recipe for recipe, _ in results],
        duration_ns=(datetime.now().timestamp() * 1_000_000_000) - start_time_ns,
    )

    # Fake an LLM response listing the results
    recipes = "\n".join([recipe["name"] for recipe, _ in results])

    response = f"""
    Recipes:
    {recipes}
    """

    logger.add_llm_span(
        name="LLM Response",
        input=query,
        output=response,
        model="gpt-5-mini",
        duration_ns=200_000_000,
        num_input_tokens=50,
        num_output_tokens=150,
        total_tokens=200,
    )

    return response

def recipe_chatbot(query: str) -> str:
    """
    Simulate a recipe chatbot interaction.
    """
    print(f"User -> {query}")
    
    # Initialize the Galileo context to log this process
    galileo_context.init(project=PROJECT_NAME, log_stream="hw4-recipe-chatbot")

    # Start a trace
    logger = galileo_context.get_logger_instance()
    logger.start_trace(input=query)

    response = process_query(query)

    # End and flush the trace
    logger.conclude(output=response)
    logger.flush()

    print(f"Bot -> {response}")

    return response
    
# Test the process function
test_query = queries[0]
recipe_chatbot(test_query['query'])

# Show Galileo information after the response
config = GalileoPythonConfig.get()
logger = galileo_context.get_logger_instance()
project_url = f"{config.console_url}project/{logger.project_id}"
log_stream_url = f"{project_url}/log-streams/{logger.log_stream_id}"

print()
print("🚀 GALILEO LOG INFORMATION:")
print(f"🔗 Project   : {project_url}")
print(f"📝 Log stream: {log_stream_url}")

Follow the link to the Log stream, and open the first trace. This will show the query, the BM25 retriever, and the response from the LLM.

![The trace with a retriever span and LLM span](./images/retriever-trace.webp)

## Step 3: Evaluate retrieval

We can now evaluate the retrieval using a metric. In this case, we can use a code-based metric, instead of an LLM-as-a-judge metric, as we have access to a ground truth to look up how successful the retrieval was.

We can do this using a [local metric](https://v2docs.galileo.ai/concepts/metrics/custom-metrics/custom-metrics-ui-code#local-metrics). This is a metric we can use in an experiment that runs locally. This makes our metric simpler. You can take the local metric and upload it to Galileo later if needed.

For this example, we'll just measure the mean reciprocal rank, but you can use the same idea to measure the different Recall@k values.

First we need to add the queries to a dataset in Galileo so that our experiments are reproducible.

In [ ]:
from galileo.datasets import create_dataset, get_dataset

# First check if we already have the dataset. If not, create it.
DATASET_NAME = "hw4-recipe-chatbot-queries"
dataset = get_dataset(name=DATASET_NAME)

if dataset is None:
    dataset = create_dataset(
        name=DATASET_NAME,
        content=[
            {
                "input": q["query"],
                "output": json.dumps(q),
            } for q in queries
        ],
    )

print(f"Dataset: {os.environ.get('GALILEO_CONSOLE_URL', 'https://app.galileo.ai/').removesuffix('/')}/datasets/{dataset.id}")

The dataset values are passed through the experiment to the retriever span, so we can access the raw ground truth data from the dataset information. This contains the `source_recipe_id` that we can retrieve and use to find the rank of that recipe in the retrieved output.

In [ ]:
from galileo import Trace, Span, StepType
from galileo.schema.metrics import LocalMetricConfig

def score_retriever_span(step: Span | Trace) -> float:
    """
    Evaluate the Mean Reciprocal Rank (MRR) for the retriever step.
    """
    # Look up the query in the dataset
    dataset_output = json.loads(step.dataset_output)
    ground_truth_recipe_id = dataset_output['source_recipe_id']
    
    retrieved_ids = [json.loads(r.content)["id"] for r in step.output]
    
    if ground_truth_recipe_id in retrieved_ids:
        rank = retrieved_ids.index(ground_truth_recipe_id) + 1
        reciprocal_rank = 1.0 / rank
    else:
        reciprocal_rank = 0.0

    return reciprocal_rank

def score_retriever_aggregator(ranks: list[float]) -> float:
    """
    Average the scores. This is used to get an average score across a trace.
    """
    return sum(ranks) / len(ranks)

# Create a local metric config that we can pass to the experiment
mean_reciprocal_rank = LocalMetricConfig[float](
    name="Mean Reciprocal Rank",
    scorer_fn=score_retriever_span,
    aggregator_fn=score_retriever_aggregator,
    scorable_types=[StepType.retriever],
    aggregatable_types=[StepType.trace]
)

Now we can run an experiment against the dataset, using our simple 'chatbot' function that incorporates the retriever.

In [ ]:
from galileo.experiments import run_experiment

experiment = run_experiment(
    experiment_name="hw4-recipe-chatbot-experiment",
    project=PROJECT_NAME,
    dataset=DATASET_NAME,
    function=process_query,
    metrics=[mean_reciprocal_rank],
)

print(f"🔗 Experiment : {experiment['link']}")

This code prints out a link to the experiment. You can follow the link to see the scores.

![Rows in the experiment with calculated MRR](./images/calculated-mrr.webp)